<a href="https://colab.research.google.com/github/JeyTee1988/LLM-Schulung/blob/main/TransformerVerstehen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wichtig**: Wenn Du dieses Notebook in Google Colab ausführst, wähle zuerst unter "Laufzeit > Laufzeittyp ändern" eine GPU aus, um die Ausführung zu beschleunigen.

# Transformer verstehen

Nun schauen wir in die Pipeline rein, um zu verstehen, wie Transformer funktionieren.

Die Pipeline stellt unser gesamtes Language Model dar. Sie besteht bei Transformern aus einem Tokenizer und dem eigentlichen Neuronalen Netz.

In [1]:
!pip install transformers einops accelerate

In [2]:
from transformers import pipeline

pipe = pipeline("text-generation", model="distilgpt2")

tokenizer = pipe.tokenizer
model = pipe.model

## Tokenizer

Der Tokenizer in einem (Large) Language Model (LLM) dient dazu, Text in kleinere Einheiten, sogenannte Tokens, zu zerlegen. Diese Tokens können einzelne Wörter, Phrasen oder sogar einzelne Buchstaben sein.

Der Tokenizer ist wichtig, weil ein LLM wie ein neuronales Netzwerk arbeitet, das auf Zahlen, nicht auf Wörtern, basiert. Es kann nicht direkt Text verstehen oder interpretieren. Stattdessen muss es den Text in eine numerische Form umwandeln, die es verarbeiten kann.

Dieser Prozess des Zerlegens und Umwandeln wird durch den Tokenizer durchgeführt. Er nimmt den ursprünglichen Text, teilt ihn in Tokens und wandelt diese dann in Zahlen um, die das Modell verarbeiten kann.

https://platform.openai.com/tokenizer


In [3]:
text = "We are happy to have you participate in our modest LLM workshop!"
tokenizer.tokenize(text)

['We',
 'Ġare',
 'Ġhappy',
 'Ġto',
 'Ġhave',
 'Ġyou',
 'Ġparticipate',
 'Ġin',
 'Ġour',
 'Ġmodest',
 'ĠLL',
 'M',
 'Ġworkshop',
 '!']

In [4]:
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input.input_ids

tensor([[ 1135,   389,  3772,   284,   423,   345,  8277,   287,   674, 12949,
         27140,    44, 20243,     0]])

Der Tokenizer enthält ein Dictionary, welches jedem Token eine Zahl zuweist

In [5]:
vocab = tokenizer.get_vocab()
first_20_items = {k: vocab[k] for k in list(vocab.keys())[:20]}
first_20_items

{'Ġ372': 46633,
 'Ġwarns': 22145,
 'Ġsaturated': 24725,
 'ĠVo': 20687,
 'Lind': 43410,
 'Ġcaregivers': 45044,
 'ĠAux': 47105,
 'Ġclears': 37526,
 'Guest': 42481,
 'Ġtimestamp': 41033,
 'ĠAdamant': 42565,
 'Ġculp': 21286,
 'Ġreminis': 21484,
 'Ġmanufact': 4011,
 'Ġdwarf': 24603,
 '727': 47760,
 'el': 417,
 'Ġuntil': 1566,
 'Ġunexpected': 10059,
 'ĠStadium': 10499}

Es gibt insgesamt 50257 Tokens in diesem Tokenizer.

In [6]:
len(vocab)

50257

## Transformer-Modell

Das Modell ist der eigentliche Transformer, der den encodierten Text entgegen nimmt und verarbeitet oder *transformiert* 😎.

### Die Ausgabe verstehen

Statt die Pipeline als ganzes zu nutzen, können wir auch manuell die Ausgabe des Encoders durch unser Modell schleusen, um zu sehen, was es damit macht.

Obwohl wir nur den fehlenden Text vervollständigen wollen, liefert unser Transformer tatsächlich für **jedes** Token in unserem Input-Text eine Vorhersage für das nächste Token:

In [7]:
output = model(**encoded_input)
token_logits = output.logits

# Get the predicted token ids
predicted_token_ids = token_logits.argmax(dim=-1)
predicted_token_ids

tensor([[  383,   407,   284,  5453,   262,   319,   287,   262,  7865, 18066,
         15996,  1628,    13,   198]])

In [8]:
# Decode the tokens to text
predicted_text = [tokenizer.decode(t) for t in predicted_token_ids]

for index, t in enumerate(predicted_token_ids[0]):
  input_token = tokenizer.tokenize(text)[index]
  predicted_next_token = tokenizer.decode(t)
  print(input_token + " => " + predicted_next_token)


We =>  The
Ġare =>  not
Ġhappy =>  to
Ġto =>  announce
Ġhave =>  the
Ġyou =>  on
Ġparticipate =>  in
Ġin =>  the
Ġour =>  upcoming
Ġmodest =>  fundraising
ĠLL => VM
M =>  project
Ġworkshop => .
! => 



Das liegt daran, dass Transformer auf massiv parallele Verarbeitung und paralleles Training ausgelegt sind. Deswegen verarbeiten sie immer den gesamten Text parallel.

Um eine größere Spanne von Texten erzeugen zu können, nutzen Transformer nicht immer die wahrscheinlichste Vorhersage (Stichwort **Temperatur**).

Hier sind die fünf wahrscheinlichsten Vorhersagen:

In [9]:
import torch

# Convert logits to probabilities
token_probs = torch.nn.functional.softmax(token_logits, dim=-1)

# Get the top 5 predicted token ids
top_k = 5
top_k_probs, top_k_indices = token_probs.topk(top_k, dim=-1)

# Iterate over the tokens and print the top 5 predictions for each one
for i in range(top_k_indices.shape[1]):
    input_token = tokenizer.tokenize(text)[i]
    print(f"Input: {input_token}:")
    for j in range(top_k):
        token_id = top_k_indices[0, i, j].item()
        probability = top_k_probs[0, i, j].item()
        output_token = tokenizer.decode([token_id])
        print(f"    Output: '{output_token}', probability: {probability:.2%}")


Input: We:
    Output: ' The', probability: 4.03%
    Output: ' A', probability: 1.25%
    Output: 'The', probability: 1.19%
    Output: '
', probability: 1.12%
    Output: 'I', probability: 1.12%
Input: Ġare:
    Output: ' not', probability: 6.18%
    Output: ' going', probability: 2.36%
    Output: ' very', probability: 2.28%
    Output: ' all', probability: 2.03%
    Output: ' looking', probability: 1.96%
Input: Ġhappy:
    Output: ' to', probability: 75.27%
    Output: ' that', probability: 9.01%
    Output: ' with', probability: 5.26%
    Output: ' and', probability: 1.83%
    Output: ' we', probability: 1.26%
Input: Ġto:
    Output: ' announce', probability: 55.91%
    Output: ' be', probability: 4.09%
    Output: ' have', probability: 3.21%
    Output: ' share', probability: 3.20%
    Output: ' welcome', probability: 2.78%
Input: Ġhave:
    Output: ' the', probability: 10.51%
    Output: ' been', probability: 6.17%
    Output: ' a', probability: 5.33%
    Output: ' you', probabi

### Die Architektur verstehen


In [10]:
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Laden des vortrainierten Modells und des zugehörigen Tokenizers
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2', output_hidden_states=True)

# GPU nutzen wenn verfügar
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device).eval()

text = 'Das ist ein Beispieltext für die Demonstration.'

tokens = tokenizer.tokenize(text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)

# Verfolgen der Veränderungen in den latenten Zuständen und Bestätigen der konstanten Dimension.
# Dazu führen wir die Eingabe durch das gesamte Modell und drucken die Größe der Ausgabe nach jedem Block
input_tensor = torch.tensor([input_ids], dtype=torch.long).unsqueeze(0).to(device)

with torch.no_grad():
    # Durchführen einer Vorwärtsdurchlaufe durch das Modell
    outputs = model(input_tensor)
    # Die Ausgabe ist ein Tuple, das die Endausgabe und eine Liste der Zwischenausgaben enthält
    final_output, hidden_states = outputs.last_hidden_state, outputs.hidden_states
    # Drucken Sie die Größe der Endausgabe und jeder Zwischenausgabe
    print(f"Größe der Endausgabe:\n\t{final_output.size()}\n")
    print(f"Endausgabe:\n\t{final_output}\n")
    for i, hidden_state in enumerate(hidden_states):
        print(f"Größe der latenten Zustände nach Block {i+1}:\n\t{hidden_state.size()}\n")
        print(f"Latente Zustände nach Block {i+1}:\n\t{hidden_state}\n\n")

Größe der Endausgabe:
	torch.Size([1, 1, 16, 768])

Endausgabe:
	tensor([[[[-0.1805, -0.1602, -0.2445,  ..., -0.1593,  0.0039, -0.0136],
          [-0.3021, -0.3064, -0.4753,  ...,  0.0716, -0.1752,  0.2778],
          [-0.6312, -0.3724, -0.7612,  ...,  0.3610, -0.1038,  0.0860],
          ...,
          [ 0.6071, -0.1206, -0.9526,  ..., -0.4354,  0.0310, -0.5339],
          [ 0.0030, -0.4362, -0.2058,  ..., -0.1486, -0.3879, -0.3260],
          [ 0.0698, -0.4597, -0.5089,  ..., -0.2768, -0.2152, -0.0472]]]],
       device='cuda:0')

Größe der latenten Zustände nach Block 1:
	torch.Size([1, 16, 768])

Latente Zustände nach Block 1:
	tensor([[[ 0.0911, -0.1488,  0.1890,  ..., -0.0553,  0.1373, -0.0367],
         [-0.0502, -0.2643, -0.0026,  ...,  0.1705, -0.0574, -0.0334],
         [-0.0055, -0.0747,  0.1101,  ...,  0.1342, -0.0187, -0.0468],
         ...,
         [-0.0501,  0.0583,  0.4026,  ..., -0.0276, -0.2199, -0.0523],
         [-0.0544,  0.0188,  0.1011,  ..., -0.0354, -0.1995, 